In [ ]:
import tensorflow as tf
print(tf.__version__)

2.15.0


# Nhập thư viện

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
import tensorflow.keras.backend as K

In [ ]:
# physical_devices = tf.config.list_physical_devices("GPU")
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

#Lấy và Show Data

In [ ]:
import os
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import re

In [ ]:
def preprocess_and_convert_to_list(series):
    # Chuyển đổi các chuỗi thành viết thường và loại bỏ dấu câu
    def preprocess_text(text):
        text_lower = text.lower()  # Chuyển thành lowercase
        text_no_punctuation = re.sub(r'[^\w\s]', '', text_lower)  # Loại bỏ dấu câu
        return text_no_punctuation

    # Áp dụng hàm xử lý văn bản cho từng phần tử trong Series
    processed_series = series.apply(preprocess_text)

    # Chuyển đổi Series thành danh sách
    list_strings = processed_series.tolist()

    return list_strings

In [ ]:
file_train = '/content/drive/MyDrive/All_CSV/NAML/MINDsmall_train'

path_new4train = os.path.join(file_train, 'news.tsv')
df_new4train = pd.read_csv(path_new4train, delimiter='\t', header=None)
df_new4train.fillna('', inplace=True)
df_new4train.head(5)

,0,1,2,3,4,5,6,7
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ..."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://assets.msn.com/labs/mind/AAAKEkt.html,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."


# Định nghĩa Hyperparameter

In [ ]:
title_length = 20
title_dim = 100
title_vocab = 20000
body_length = 100
body_dim = 200
body_vocab = 60000
filters = 300
kernel_size = 3
query_vector_dim = 200
category_vocab = len(df_new4train.iloc[:,1].unique())
category_length = 1
category_dim = 50
subcategory_vocab = len(df_new4train.iloc[:,2].unique())
subcategory_length = 1
TOTAL_BROWSE_NEW = 30
TOTAL_CANDIDATE_NEW = 5
batch_size = 32
num_epochs = 2

# Làm sạch data, tạo từ điển

In [ ]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Lấy data Train
categories_train = df_new4train.iloc[:,1].tolist()
subcategories_train = df_new4train.iloc[:,2].tolist()
titles_train = preprocess_and_convert_to_list(df_new4train.iloc[:,3])
bodies_train = preprocess_and_convert_to_list(df_new4train.iloc[:,4])


In [ ]:
onehot_title_train = [one_hot(words, title_vocab)for words in titles_train]
onehot_body_train = [one_hot(words, body_vocab)for words in bodies_train]
onehot_category_train = [one_hot(words, category_vocab)for words in categories_train]
onehot_subcategory_train = [one_hot(words, subcategory_vocab)for words in subcategories_train]


In [ ]:
titles_train_docs = pad_sequences(onehot_title_train, padding='pre', maxlen=title_length)
bodies_train_docs = pad_sequences(onehot_body_train, padding='pre', maxlen=body_length)
category_train_docs = pad_sequences(onehot_category_train, padding='pre', maxlen=category_length)
subcategory_train_docs = pad_sequences(onehot_subcategory_train, padding='pre', maxlen=subcategory_length)


In [ ]:
# Tạo từ điển news trong đó key là mã bài báo, value là danh sách các thông tin của new [category, sub_category, title, body]
news_dict1 = {}
for i in range(len(titles_train_docs)):
    news_dict1[df_new4train[0][i]] = [category_train_docs[i], subcategory_train_docs[i], titles_train_docs[i], bodies_train_docs[i]]

# Dữ liệu hành vi

In [ ]:
path_behavior4train = os.path.join(file_train, 'behaviors.tsv')
df_behavior4train = pd.read_csv(path_behavior4train, delimiter='\t', header=None)
df_behavior4train.fillna('', inplace=True)
df_behavior4train.head(5)

,0,1,2,3,4
0,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0
1,2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...
2,3,U73700,11/14/2019 7:01:48 AM,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...
3,4,U34670,11/11/2019 5:28:05 AM,N45729 N2203 N871 N53880 N41375 N43142 N33013 ...,N35729-0 N33632-0 N49685-1 N27581-0
4,5,U8125,11/12/2019 4:11:21 PM,N10078 N56514 N14904 N33740,N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...


# Tạo dữ liệu train, test

In [ ]:
def make_data4train(df=df_behavior4train):
    y=[]
    list_category_c=[]
    list_subcategory_c=[]
    list_title_c=[]
    list_body_c=[]
    list_category_b=[]
    list_subcategory_b=[]
    list_title_b=[]
    list_body_b=[]
    df.index = range(1, len(df) + 1)
    for i, user in df.iterrows():
        # Xử lý candidate_news
        c_c = []
        sc_c = []
        t_c = []
        b_c = []
        pos = []
        neg = []
        yy = []
        for candidate_new in user[4].split(): # Duyệt các cadidate
            candidate_split = candidate_new.split('-')
            if int(candidate_split[1]) == 1:
                pos.append(candidate_new)
            else:
                neg.append(candidate_new)
        my_list = [pos[0]] + neg[:min(len(neg), TOTAL_CANDIDATE_NEW - 1)] + ['none'] * (TOTAL_CANDIDATE_NEW - 1 - min(len(neg), 4))
        # Trộn các bài candidate_new
        random.shuffle(my_list)
        for candidate_new in my_list:
            candidate_split = candidate_new.split('-')
            if candidate_split[0] != 'none':
                data = news_dict1[candidate_split[0]]
                yy.append(int(candidate_split[1]))
                c_c.append(data[0])
                sc_c.append(data[1])
                t_c.append(data[2])
                b_c.append(data[3])
            else:
                yy.append(int(0))
                c_c.append(np.zeros(1))
                sc_c.append(np.zeros(1))
                t_c.append(np.zeros(title_length))
                b_c.append(np.zeros(body_length))
        list_category_c.append(c_c)
        list_subcategory_c.append(sc_c)
        list_title_c.append(t_c)
        list_body_c.append(b_c)
        # Xử lý browsed_news
        c_bn = [np.zeros(1) for x in range(TOTAL_BROWSE_NEW)]
        sc_bn = [np.zeros(1) for x in range(TOTAL_BROWSE_NEW)]
        t_bn = [np.zeros(title_length) for x in range(TOTAL_BROWSE_NEW)]
        b_bn = [np.zeros(body_length) for x in range(TOTAL_BROWSE_NEW)]
        for j, bnew in enumerate(user[3].split()):
            if j < TOTAL_BROWSE_NEW:
                new = news_dict1[bnew]
                c_bn[j] = new[0]
                sc_bn[j] = new[1]
                t_bn[j] = new[2]
                b_bn[j] = new[3]
            else:
                break
        list_category_b.append(c_bn)
        list_subcategory_b.append(sc_bn)
        list_title_b.append(t_bn)
        list_body_b.append(b_bn)
        y.append(np.array(yy))
        if i % batch_size == 0:
            c_b = tf.convert_to_tensor(list_category_b)
            c_b = tf.reshape(c_b, (batch_size, TOTAL_BROWSE_NEW, 1))

            sc_b = tf.convert_to_tensor(list_subcategory_b)
            sc_b = tf.reshape(sc_b, (batch_size, TOTAL_BROWSE_NEW, 1))

            t_b = tf.convert_to_tensor(list_title_b)
            t_b = tf.reshape(t_b, (batch_size, TOTAL_BROWSE_NEW, title_length))

            b_b = tf.convert_to_tensor(list_body_b)
            b_b = tf.reshape(b_b, (batch_size, TOTAL_BROWSE_NEW, body_length))

            c_c = tf.convert_to_tensor(list_category_c)
            c_c = tf.reshape(c_c, (batch_size, TOTAL_CANDIDATE_NEW, 1))

            sc_c = tf.convert_to_tensor(list_subcategory_c)
            sc_c = tf.reshape(sc_c, (batch_size, TOTAL_CANDIDATE_NEW, 1))

            t_c = tf.convert_to_tensor(list_title_c)
            t_c = tf.reshape(t_c, (batch_size, TOTAL_CANDIDATE_NEW, title_length))

            b_c = tf.convert_to_tensor(list_body_c)
            b_c = tf.reshape(b_c, (batch_size, TOTAL_CANDIDATE_NEW, body_length))
            yield ([c_b, sc_b, t_b, b_b, c_c, sc_c, t_c, b_c], np.array(y))
            y=[]
            list_category_c=[]
            list_subcategory_c=[]
            list_title_c=[]
            list_body_c=[]
            list_category_b=[]
            list_subcategory_b=[]
            list_title_b=[]
            list_body_b=[]

In [ ]:
y_true = []
def make_data4test(df):
    global y_true
    y=[]
    list_category_c=[]
    list_subcategory_c=[]
    list_title_c=[]
    list_body_c=[]
    list_category_b=[]
    list_subcategory_b=[]
    list_title_b=[]
    list_body_b=[]
    df.index = range(1, len(df) + 1)
    for i, user in df.iterrows():
        # Xử lý candidate_news
        c_c = []
        sc_c = []
        t_c = []
        b_c = []
        pos = []
        neg = []
        yy = []
        for candidate_new in user[4].split(): # Duyệt các cadidate
            candidate_split = candidate_new.split('-')
            if int(candidate_split[1]) == 1:
                pos.append(candidate_new)
            else:
                neg.append(candidate_new)
        my_list = pos[:min(4, len(pos))] + neg[:min(4, len(neg))] + ['none'] * (8 - min(4, len(pos)) - min(4, len(neg)))
        # Trộn các bài candidate_new
        random.shuffle(my_list)
        for candidate_new in my_list:
            candidate_split = candidate_new.split('-')
            if candidate_split[0] != 'none':
                data = news_dict1[candidate_split[0]]
                y.append(int(candidate_split[1]))
                list_category_c.append(data[0])
                list_subcategory_c.append(data[1])
                list_title_c.append(data[2])
                list_body_c.append(data[3])
            else:
                y.append(int(0))
                list_category_c.append(np.zeros(1))
                list_subcategory_c.append(np.zeros(1))
                list_title_c.append(np.zeros(title_length))
                list_body_c.append(np.zeros(body_length))
        # Xử lý browsed_news
        c_bn = [np.zeros(1) for x in range(TOTAL_BROWSE_NEW)]
        sc_bn = [np.zeros(1) for x in range(TOTAL_BROWSE_NEW)]
        t_bn = [np.zeros(title_length) for x in range(TOTAL_BROWSE_NEW)]
        b_bn = [np.zeros(body_length) for x in range(TOTAL_BROWSE_NEW)]
        for j, bnew in enumerate(user[3].split()):
            if j < TOTAL_BROWSE_NEW:
                new = news_dict1[bnew]
                c_bn[j] = new[0]
                sc_bn[j] = new[1]
                t_bn[j] = new[2]
                b_bn[j] = new[3]
            else:
                break
        for _ in range(8):
            list_category_b.append(c_bn)
            list_subcategory_b.append(sc_bn)
            list_title_b.append(t_bn)
            list_body_b.append(b_bn)
        if i % 4 == 0:
            c_b = tf.convert_to_tensor(list_category_b)
            c_b = tf.reshape(c_b, (batch_size, TOTAL_BROWSE_NEW, 1))

            sc_b = tf.convert_to_tensor(list_subcategory_b)
            sc_b = tf.reshape(sc_b, (batch_size, TOTAL_BROWSE_NEW, 1))

            t_b = tf.convert_to_tensor(list_title_b)
            t_b = tf.reshape(t_b, (batch_size, TOTAL_BROWSE_NEW, title_length))

            b_b = tf.convert_to_tensor(list_body_b)
            b_b = tf.reshape(b_b, (batch_size, TOTAL_BROWSE_NEW, body_length))

            c_c = tf.convert_to_tensor(list_category_c)
            c_c = tf.reshape(c_c, (batch_size, 1))

            sc_c = tf.convert_to_tensor(list_subcategory_c)
            sc_c = tf.reshape(sc_c, (batch_size, 1))

            t_c = tf.convert_to_tensor(list_title_c)
            t_c = tf.reshape(t_c, (batch_size, title_length))

            b_c = tf.convert_to_tensor(list_body_c)
            b_c = tf.reshape(b_c, (batch_size, body_length))
            y_true = y_true + y
            yield ([c_b, sc_b, t_b, b_b, c_c, sc_c, t_c, b_c], np.array(y))
            y=[]
            list_category_c=[]
            list_subcategory_c=[]
            list_title_c=[]
            list_body_c=[]
            list_category_b=[]
            list_subcategory_b=[]
            list_title_b=[]
            list_body_b=[]

In [ ]:
# def testfunc(df):
#     y=[]
#     list_category_c=[]
#     list_subcategory_c=[]
#     list_title_c=[]
#     list_body_c=[]
#     list_category_b=[]
#     list_subcategory_b=[]
#     list_title_b=[]
#     list_body_b=[]
#     df.index = range(1, len(df) + 1)
#     for i, user in df.iterrows():
#         # Xử lý candidate_news
#         c_c = []
#         sc_c = []
#         t_c = []
#         b_c = []
#         pos = []
#         neg = []
#         yy = []
#         for candidate_new in user[4].split(): # Duyệt các cadidate
#             candidate_split = candidate_new.split('-')
#             if int(candidate_split[1]) == 1:
#                 pos.append(candidate_new)
#             else:
#                 neg.append(candidate_new)
#         my_list = pos[:min(4, len(pos))] + neg[:min(4, len(neg))] + ['none'] * (8 - min(4, len(pos)) - min(4, len(neg)))
#         # Trộn các bài candidate_new
#         random.shuffle(my_list)
#         for candidate_new in my_list:
#             candidate_split = candidate_new.split('-')
#             if candidate_split[0] != 'none':
#                 data = news_dict1[candidate_split[0]]
#                 y.append(int(candidate_split[1]))
#                 yy.append(int(candidate_split[1]))
#                 list_category_c.append(data[0])
#                 list_subcategory_c.append(data[1])
#                 list_title_c.append(data[2])
#                 list_body_c.append(data[3])
#             else:
#                 y.append(int(0))
#                 yy.append(int(0))
#                 list_category_c.append(np.zeros(1))
#                 list_subcategory_c.append(np.zeros(1))
#                 list_title_c.append(np.zeros(title_length))
#                 list_body_c.append(np.zeros(body_length))
#         yy = np.array(yy)
#         if np.sum(yy) == 0:
#             print(i)
#         # Xử lý browsed_news
#         c_bn = [np.zeros(1) for x in range(TOTAL_BROWSE_NEW)]
#         sc_bn = [np.zeros(1) for x in range(TOTAL_BROWSE_NEW)]
#         t_bn = [np.zeros(title_length) for x in range(TOTAL_BROWSE_NEW)]
#         b_bn = [np.zeros(body_length) for x in range(TOTAL_BROWSE_NEW)]
#         for j, bnew in enumerate(user[3].split()):
#             if j < TOTAL_BROWSE_NEW:
#                 new = news_dict1[bnew]
#                 c_bn[j] = new[0]
#                 sc_bn[j] = new[1]
#                 t_bn[j] = new[2]
#                 b_bn[j] = new[3]
#             else:
#                 break
#         for _ in range(8):
#             list_category_b.append(c_bn)
#             list_subcategory_b.append(sc_bn)
#             list_title_b.append(t_bn)
#             list_body_b.append(b_bn)
#         if i % 4 == 0:
#             c_b = tf.convert_to_tensor(list_category_b)
#             c_b = tf.reshape(c_b, (batch_size, TOTAL_BROWSE_NEW, 1))

#             sc_b = tf.convert_to_tensor(list_subcategory_b)
#             sc_b = tf.reshape(sc_b, (batch_size, TOTAL_BROWSE_NEW, 1))

#             t_b = tf.convert_to_tensor(list_title_b)
#             t_b = tf.reshape(t_b, (batch_size, TOTAL_BROWSE_NEW, title_length))

#             b_b = tf.convert_to_tensor(list_body_b)
#             b_b = tf.reshape(b_b, (batch_size, TOTAL_BROWSE_NEW, body_length))

#             c_c = tf.convert_to_tensor(list_category_c)
#             c_c = tf.reshape(c_c, (batch_size, 1))

#             sc_c = tf.convert_to_tensor(list_subcategory_c)
#             sc_c = tf.reshape(sc_c, (batch_size, 1))

#             t_c = tf.convert_to_tensor(list_title_c)
#             t_c = tf.reshape(t_c, (batch_size, title_length))

#             b_c = tf.convert_to_tensor(list_body_c)
#             b_c = tf.reshape(b_c, (batch_size, body_length))
#             y=[]
#             list_category_c=[]
#             list_subcategory_c=[]
#             list_title_c=[]
#             list_body_c=[]
#             list_category_b=[]
#             list_subcategory_b=[]
#             list_title_b=[]
#             list_body_b=[]

# num_sample = len(df_behavior4train[0])
# num_train = int(num_sample * 0.9)
# testfunc(df_behavior4train.iloc[num_train:])

# Model

## News Encoder

In [ ]:
from tensorflow.keras import layers

### Title

In [ ]:
class AttetionBlock(Layer):
    def __init__(self, **kwargs):
        super(AttetionBlock, self).__init__(**kwargs)
        self.dense1 = Dense(200, activation='tanh')
        self.dense2 = Dense(1)
        self.flatten = Flatten()
        self.activation = Activation('softmax')
        self.dot = Dot((1, 1))

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.flatten(x)
        W = self.activation(x)
        outputs = self.dot([inputs, W])
        return outputs

In [ ]:
class TitleBlock(Layer):
    def __init__(self, vocab_size=title_vocab, vector_dim=title_dim, sent_length=title_length):
        super(TitleBlock, self).__init__()
        self.embedding = Embedding(input_dim=vocab_size, output_dim=vector_dim, input_length=sent_length)
        self.dropout = Dropout(0.2)
        self.conv = Conv1D(filters=300, kernel_size=3, padding='same')
        self.bn = BatchNormalization()
        self.attention = AttetionBlock()

    def call(self, input_onehot):
        x = self.embedding(input_onehot)
        x = self.dropout(x)
        x = self.conv(x)
        x = self.dropout(x)
        x = self.bn(x)
        x = self.attention(x)
        return x

### Body

In [ ]:
class BodyBlock(layers.Layer):
    def __init__(self, vocab_size=body_vocab, vector_dim=body_dim, sent_length=body_length):
        super(BodyBlock, self).__init__()
        self.embedding = Embedding(input_dim=vocab_size, output_dim=vector_dim, input_length=sent_length)
        self.dropout = Dropout(0.2)
        self.conv = Conv1D(filters=300, kernel_size=3, padding='same')
        self.bn = BatchNormalization()
        self.attention = AttetionBlock()

    def call(self, input_onehot):
        x = self.embedding(input_onehot)
        x = self.dropout(x)
        x = self.conv(x)
        x = self.dropout(x)
        x = self.bn(x)
        x = self.attention(x)
        return x

### Categories

In [ ]:
class CategoryBlock(layers.Layer):
    def __init__(self, vocab_size=category_vocab, vector_dim=category_dim, sent_length=category_length):
        super(CategoryBlock, self).__init__()
        self.embedding = Embedding(input_dim=vocab_size, output_dim=vector_dim, input_length=sent_length)
        self.dense = Dense(300, activation='relu') # = filters

    def call(self, input):
        x = self.embedding(input)
        x = self.dense(Flatten()(x))
        return x

### Attention Pooling

In [ ]:
class Attention_Pooling(Layer):
    def __init__(self):
        super(Attention_Pooling, self).__init__()
        self.dense1 = Dense(200, activation='tanh')
        self.dense2 = Dense(1)
        self.lamda = Lambda(lambda x:K.squeeze(x,axis=-1))
        self.activation = Activation('softmax')
        self.dot = Dot((1, 1))

    def call(self, inputs):
        x = self.dense1(inputs)
        W = self.dense2(x)
        W = self.lamda(W)
        W = self.activation(W)
        outputs = self.dot([inputs, W])
        return outputs

In [ ]:
class NewEncoder(Model):
    def __init__(self):
        super(NewEncoder, self).__init__()
        self.title = TitleBlock()
        self.body = BodyBlock()
        self.category = CategoryBlock()
        self.sub_category = CategoryBlock(vocab_size=subcategory_vocab, vector_dim=category_dim, sent_length=category_length)
        self.attention = Attention_Pooling()

    def call(self, input_new):
        category = self.category(input_new[0])
        sub_category = self.sub_category(input_new[1])
        title = self.title(input_new[2])
        body = self.body(input_new[3])
        sample = [category, sub_category, title, body]
        sample = Concatenate(axis=1)([K.expand_dims(x,axis=1) for x in sample])
        x = self.attention(sample)
        return x

## User Encoder

In [ ]:
class UserEncoder(Layer): # Đầu vào là list browednews của từng
    def __init__(self, newEncoder):
        super(UserEncoder, self).__init__()
        self.newEncoder = newEncoder
        self.attention = AttetionBlock()

    def call(self, input): # 32 4 30 x
        browsednews = [self.newEncoder([input[0][i], input[1][i], input[2][i], input[3][i]]) for i in range(batch_size)] # 32 x (30, 200)
        browsednews_matrix = concatenate([Lambda(lambda x: K.expand_dims(x,axis=0))(news) for news in browsednews],axis=0)
        x = self.attention(browsednews_matrix) # 32, 300
        return x


## Click Predictor

In [ ]:
class ClickPredictor(Layer): # for train
    def __init__(self):
        super(ClickPredictor, self).__init__()
        self.activation = Activation('softmax')

    def call(self, browsed_news, candidate_news): #(32, 200) and 32x(5,200)
        browsed_news = tf.reshape(browsed_news, (batch_size, filters)) # (32, 200)
        candidate_news = tf.reshape(candidate_news, (batch_size, TOTAL_CANDIDATE_NEW, filters)) # (32, 5, 200)
        candidate_news = [candidate_news[:,i,:] for i in range(candidate_news.shape[1])] # 5 x (32, 200)
        x = [keras.layers.dot([browsed_news, candidate], axes=-1) for candidate in candidate_news]
        x = keras.layers.concatenate(x) # (32, 5)
        x = self.activation(x)
        return x

In [ ]:
class ClickScore(Layer): # for test
    def __init__(self):
        super(ClickScore, self).__init__()
        self.activation = Activation('sigmoid')

    def call(self, browsed_news, candidate): #(32, 200) and (32, 200)
        browsed_news = tf.reshape(browsed_news, (batch_size, filters)) # (32, 200)
        candidate = tf.reshape(candidate, (batch_size, filters)) # (32, 200)
        x = keras.layers.dot([browsed_news, candidate], axes=-1)
        x = tf.squeeze(x)
        x = self.activation(x)
        return x

# Trainning

In [ ]:
class ModelTrain(Model):
    def __init__(self, newEncoder):
        super(ModelTrain, self).__init__()
        self.newEncoder = newEncoder
        self.user_encoder = UserEncoder(self.newEncoder)
        self.click_predictor = ClickPredictor()

    def call(self, inputs): # 32, 5, 20
        user_encoder = self.user_encoder([inputs[0], inputs[1], inputs[2], inputs[3]]) # 32 x (200)
        candidate_encoders = [self.newEncoder([inputs[4][i], inputs[5][i], inputs[6][i], inputs[7][i]]) for i in range(batch_size)] # 32 x (5, 200)
        outputs = self.click_predictor(user_encoder, candidate_encoders)
        return outputs

In [ ]:
class ModelTest(Model):
    def __init__(self, newEncoder):
        super(ModelTest, self).__init__()
        self.newEncoder = newEncoder
        self.user_encoder = UserEncoder(self.newEncoder)
        self.click_score = ClickScore()

    def call(self, inputs): # 32, 5, 20
        user_encoder = self.user_encoder([inputs[0], inputs[1], inputs[2], inputs[3]]) # 32 x (200)
        candidate = self.newEncoder([inputs[4], inputs[5], inputs[6], inputs[7]]) # (32, 200)
        outputs = self.click_score(user_encoder, candidate) # (32, )
        return outputs

In [ ]:
newEncoder = NewEncoder()
model_train = ModelTrain(newEncoder)
model_train.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=["accuracy"]
)

n = len(df_behavior4train[0])
num_sample = int(n * 0.1)
df_beta = df_behavior4train.iloc[:num_sample]
num_train = int(len(df_beta[0]) * 0.95)
for ep in range(3):
    data_train = make_data4train(df_beta.iloc[:num_train])
    model_train.fit_generator(data_train, epochs=1, steps_per_epoch=num_train//batch_size)

In [ ]:
data_test = make_data4test(df_beta.iloc[num_train:])
num_test = int(len(df_beta.iloc[num_train:][0]))
model_test = ModelTest(newEncoder)
y_pred = model_test.predict_generator(data_test, steps=num_test//4,verbose=2)
y_true = np.array(y_true)

<ipython-input-40-eba0c610616a>:4: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  y_pred = model_test.predict_generator(data_test, steps=num_test//4,verbose=2)


196/196 - 239s - 239s/epoch - 1s/step


In [ ]:
from sklearn.metrics import roc_auc_score
def dcg_score(y_true, y_score, k=10):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])
    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gains / discounts)


def ndcg_score(y_true, y_score, k=10):
    best = dcg_score(y_true, y_true, k)
    actual = dcg_score(y_true, y_score, k)
    return actual / best


def mrr_score(y_true, y_score):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order)
    rr_score = y_true / (np.arange(len(y_true)) + 1)
    return np.sum(rr_score) / np.sum(y_true)

In [ ]:
all_auc=[]
all_mrr=[]
all_ndcg5=[]
all_ndcg10=[]
for i in range(num_test-4):
    all_auc.append(roc_auc_score(y_true[i*8:(i+1)*8], y_pred[i*8:(i+1)*8]))
    all_mrr.append(mrr_score(y_true[i*8:(i+1)*8], y_pred[i*8:(i+1)*8]))
    all_ndcg5.append(ndcg_score(y_true[i*8:(i+1)*8], y_pred[i*8:(i+1)*8], k=5))
    all_ndcg10.append(ndcg_score(y_true[i*8:(i+1)*8], y_pred[i*8:(i+1)*8], k=10))
print(np.mean(all_auc),np.mean(all_mrr),np.mean(all_ndcg5),np.mean(all_ndcg10))

0.30344643619291506 0.19725715403532307 0.21216026615931363 0.41952486422032925
